# 对象的比较、拷贝


```python
l1 = [1, 2, 3]
l2 = list(l1)
```

这些语句的背后发生了什么？

- l2 是 l1 的浅拷贝（shallow copy）还是深度拷贝（deep copy）呢？
- a == b是比较两个对象的值相等，还是两个对象完全相等呢？

   - '=='操作符比较对象之间的值是否相等
   - 'is'操作符比较的是对象的身份标识是否相等，即它们是否是同一个对象，是否指向同一个内存地址。
   - 在 Python 中，每个对象的身份标识，都能通过函数 id(object) 获得。
   

In [11]:
a = 200
b = 200

print(a == b)
print(id(a))
print(id(b))
print(a is b)

True
4532661600
4532661600
True


对于整型数字来说，以上 a is b 为 True 的结论，只适用于 -5 到 256 范围内的数字。这是因为 Python 内部会对 -5 到 256 的整型维持一个数组，起到一个缓存的作用。每次你试图创建一个 -5 到 256 范围内的整型数字时，Python 都会从这个数组中返回相对应的引用，而不是重新开辟一块新的内存空间。

In [12]:
a = 257
b = 257

print(a == b)
print(id(a))
print(id(b))
print(a is b)

True
140563284206960
140563284206736
False


比较操作符'is'的效率，通常要优于'=='。因为'is'操作符不能被重载，这样，Python 就不需要去寻找，程序中是否有其他地方重载了比较操作符，并去调用。

但是'=='操作符却不同，执行 a == b 相当于是去执行 a.\__eq\__(b)，而 Python 大部分的数据类型都会去重载
\__eq\__ 这个函数，其内部的处理通常会复杂一些。比如，对于列表，\__eq\__函数会去遍历列表中的元素，比较它们的顺序和值是否相等。


In [13]:
# 对于不可变（immutable）的变量

t1 = (1, 2, [3, 4])
t2 = (1, 2, [3, 4])
print(t1 == t2)

t1[-1].append(5)
print(t1 == t2)

True
False


## 浅拷贝和深度拷贝

浅拷贝（shallow copy）和深度拷贝（deep copy）。
- 浅拷贝，是指重新分配一块内存，创建一个新的对象，里面的元素是原对象中子对象的引用（指原对象内第一层对象的引用）。因此，如果原对象中的元素不可变，那倒无所谓；但如果元素可变，浅拷贝通常会带来一些副作用，尤其需要注意。
- 如果我们想避免这种副作用，完整地拷贝一个对象，你就得使用深度拷贝。
- 深度拷贝，是指重新分配一块内存，创建一个新的对象，并且将原对象中的元素，以递归的方式，通过创建新的子对象拷贝到新对象中。（相对于浅拷贝，深度拷贝的时间和空间开销要高多了）

常见的浅拷贝的方法，

- 使用数据类型本身的构造器（工厂函数）

In [15]:
l1 = [1, 2, 3]
l2 = list(l1)

print(l2)
print(l1 == l2)
print(l1 is l2)

s1 = set([1, 2, 3])
s2 = set(s1)

print(s2)
print(s1 == s2)
print(s1 is s2)

[1, 2, 3]
True
False
{1, 2, 3}
True
False


- 切片操作符':'完成浅拷贝

In [16]:
l1 = [1, 2, 3]
l2 = l1[:]

print(l2)
print(l1 == l2)
print(l1 is l2)

[1, 2, 3]
True
False


>需要注意的是，对于元组，使用 tuple() 或者切片操作符':'不会创建一份浅拷贝，相反，它会返回一个指向相同元组的引用

In [18]:
t1 = (1, 2, 3)
t2 = tuple(t1)

print(t1 == t2)
print(t1 is t2)

True
True


- copy.copy 完成浅拷贝

In [17]:
import copy

l1 = [1, 2, 3]
l2 = copy.copy(l1)

print(l2)
print(l1 == l2)
print(l1 is l2)

[1, 2, 3]
True
False


深拷贝只有一种形式，copy 模块中的 deepcopy()函数。

深度拷贝也不是完美的，往往也会带来一系列问题。如果被拷贝对象中存在指向自身的引用，那么程序很容易陷入无限循环：

In [19]:
import copy

x = [1]
x.append(x)

print(x)

y = copy.deepcopy(x)
y

[1, [...]]


[1, [...]]

上面这段代码，深度拷贝 x 到 y 后，程序并没有出现 stack overflow 的现象。这是为什么呢？

这是因为深度拷贝函数 deepcopy 中会维护一个字典，记录已经拷贝的对象与其 ID。拷贝过程中，如果字典里已经存储了将要拷贝的对象，则会从字典直接返回，看下面源码：

```python
def deepcopy(x, memo=None, _nil=[]):
    """Deep copy operation on arbitrary Python objects.

    See the module's __doc__ string for more info.
    """

    if memo is None:
        memo = {}

    d = id(x)
    y = memo.get(d, _nil)   # 查询字典里是否已经存储了该对象
    if y is not _nil:
        return y    # 如果字典里已经存储了将要拷贝的对象，则直接返回

    # ...
```

In [20]:
?copy.deepcopy   

Signature: copy.deepcopy   (x, memo=None, _nil=[])
Docstring:
Deep copy operation on arbitrary Python objects.

See the module's __doc__ string for more info.
File:      ~/miniconda3/lib/python3.7/copy.py
Type:      function


In [ ]:
!cat ~/miniconda3/lib/python3.7/copy.py

## 思考题

我曾用深度拷贝，拷贝过一个无限嵌套的列表。那么，当我们用等于操作符'=='进行比较时，输出会是什么呢？是 True 或者 False 还是其他？为什么呢？

In [1]:
import copy

x = [1]
x.append(x)
y = copy.deepcopy(x)

print(x == y)
print(x is y)

RecursionError: maximum recursion depth exceeded in comparison

答：x，y都是一个无限嵌套的列表，因此会抛出 stack overflow，可以设置最深递归层次，比如1000来避免。

```python
import sys
sys.setrecursionlimit(1000)
# get the recursion limit
# sys.getrecursionlimit()
```

## 附注

如果是非容器的原子类型，不存在浅/深拷贝，如下：

```python
x = 1000

# y = copy.copy(x)
# y = copy.deepcopy(x)
# 都等同于
y = x

x== y # True
x is y # True
```